In [1]:
import torch
import numpy as np
from kvae_model import KVAEModel
from kvae import KVAE
from auxiliary import load_bouncing_ball, export_vid, export_latent_space_vis
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
batch_size = 64
train_dataset, val_dataset =  load_bouncing_ball("nonlinear_ball_data", "circle", singular=False)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle = True, num_workers = 0)
val_dataloader   = torch.utils.data.DataLoader(val_dataset,   batch_size=batch_size, shuffle = True, num_workers = 0)

## Load pretrained model

In [4]:
model = KVAEModel(x_dim = 32 * 32, 
            a_dim = 4,
            z_dim = 4, 
            x_2d=True,
            init_kf_mat = 0.05,
            noise_transition = 0.08,
            noise_emission = 0.03,
            init_cov = 20,
            K = 1,
            dim_RNN_alpha = 50,
            num_RNN_alpha = 2,
            dropout_p = 0,
            scale_reconstruction = 0.3,
            device='cpu').to('cpu')
model.build()

kvae = KVAE(model=model,
            lr = 3e-3,#3e-6,
            lr_tot = 1e-3,#1e-6,
            epochs = 20,
            batch_size = batch_size,
            early_stop_patience = 20,
            save_frequency = 1,
            only_vae_epochs = 5,
            kf_update_epochs = 5,
            save_dir = "results")

In [5]:
kvae.model.load_state_dict(torch.load("results/t25_circle_IS_G_a4/model_at_93.pt", map_location='cpu'))

<All keys matched successfully>

## Extract results from a training sample

In [19]:
export_vid(kvae, train_dataloader, j=0, both=True, epoch=555)

torch.Size([64, 32, 32, 200]) (64, 32, 32, 200)


In [31]:
t = torch.Tensor(train_dataloader.dataset[0:0+64])
kvae.model.eval()
with torch.no_grad():
    x, y, a, a_gen, mu_smooth = kvae.model.forward_debug(t, compute_loss=False)

In [32]:
# mu_smooth in seq_len, batch_size, z_dim
z_temp = mu_smooth[:, 0:1, :]
z_temp = torch.tensor(z_temp).detach()
seq_len, batch_size, _ = z_temp.shape
z_temp = z_temp.view(batch_size, kvae.model.z_dim, seq_len)

In [33]:
N = 40

In [34]:
z_samples = torch.zeros((batch_size, kvae.model.z_dim, seq_len+N))
y_samples = torch.zeros((batch_size, kvae.model.x_dim, kvae.model.x_dim, seq_len+N))
a_samples = torch.zeros((batch_size, kvae.model.a_dim, seq_len+N))

In [42]:
z = z_temp[:, :, -2:-1] #batch_size, z_dim, 1

In [43]:
K = 1

In [44]:
A_flatten = kvae.model.A.view(K, kvae.model.z_dim*kvae.model.z_dim) # (K, z_dim*z_dim) 
B_flatten = kvae.model.B.view(K, kvae.model.z_dim*kvae.model.u_dim) # (K, z_dim*u_dim) 
C_flatten = kvae.model.C.view(K, kvae.model.a_dim*kvae.model.z_dim) # (K, a_dim*z_dim) 

In [45]:
A_mix = A_flatten.view(batch_size, kvae.model.z_dim, kvae.model.z_dim)
B_mix = B_flatten.view(batch_size, kvae.model.z_dim, kvae.model.u_dim)
C_mix = C_flatten.view(batch_size, kvae.model.a_dim, kvae.model.z_dim)

In [46]:
def z_to_y(model, z, C): #z in (batch_size, z_dim, seq_len)
    model.eval()
    _, _, seq_len = z.shape
    a_gen = C.bmm(z)
    with torch.no_grad():
        y = model.decode(a_gen.view(seq_len, batch_size, kvae.model.a_dim)).permute(1,-1,0) #(batch_size, dim, seq_len)
    return a_gen, y

In [ ]:
z_samples[:, :, :seq_len] = z_temp
a_gen, y = z_to_y(kvae.mode, z_temp, C_mix)
a_samples[:, :, :seq_len] = a_gen
y_samples[:, :, :seq_len] = y

In [ ]:
# loop

In [47]:
a_gen, y = z_to_y(kvae.model, z, C_mix)

In [49]:
y.shape

torch.Size([1, 1024, 1])

In [ ]:
# Az -> z
# Cz -> a_gen
# y = decode(a_gen)

In [6]:
# TODO: make this work!!
kvae.generate(torch.tensor(mu_smooth, requires_grad=False), n_steps=20)

RuntimeError: The size of tensor a (64) must match the size of tensor b (20) at non-singleton dimension 0